# Quant track


### How to use:
- Create a virtual environment (tested on Python 3.11)
- Run the notebook (this will install all the requirements)

In [120]:
#Requirements
!pip install -r requirements.txt

In [121]:
#Imports for data analysis
import pandas as pd
import os

#Imports for Aave data fetching
#from web3 import Web3, HTTPProvider
#from eth_defi.aave_v3.constants import aave_v3_get_json_rpc_url, aave_v3_get_network_by_chain_id
#from eth_defi.event_reader.fast_json_rpc import patch_web3
#from eth_defi.aave_v3.events import aave_v3_fetch_events_to_csv
#from eth_defi.event_reader.json_state import JSONFileScanState

#Imports for currency values
from pycoingecko import CoinGeckoAPI
import datetime

#Imports for backtesting

In this notebook we are gonna analyse Aave V3 in the Polygon network for USDC, USDT, DAI and WMATIC.

For demonstrating the strategy, we will only query the first 32_000_000 blocks.

Note that a Polygon node is required. A free node can be obtained in [Infura](https://www.infura.io/).
The free API has a rate limiter, so it could take a while to download all the blocks.

In [129]:
def process_data(dataset_path):
  """
  Processes lending data from a directory structure where folders represent chains and CSV files within each folder represent currencies.

  Args:
    dataset_path: The path to the root directory of the dataset. (default: ./dataset)

  Returns:
    A dataframe with columns: timestamp, liquidityRate_avg, variableBorrowRate_avg, utilizationRate_avg, stableBorrowRate_avg, 
  """
  chains = ['polygon']
  currencies = ['USDT', 'DAI', 'SUSHI']
  all_data = []
  for chain in chains:
    chain_path = os.path.join(dataset_path, chain)
    if os.path.isdir(chain_path):
      for filename in os.listdir(chain_path):
        if filename.startswith(tuple(currencies)) and filename.endswith(".csv"):
          currency = filename[:-4]  # Remove '.csv' extension
          filepath = os.path.join(chain_path, filename)
          df = pd.read_csv(filepath)
          if df.empty: #Check if empty
            continue
          df['chain'] = chain  # Add chain column
          df['currency'] = currency  # Add currency column
          all_data.append(df)

  return pd.concat(all_data, ignore_index=True)

dataset_path = "./dataset"  
df = process_data(dataset_path)

# Sort values by timestap
df.sort_values('timestamp')

print(df)

#Calculating the spread
df['variableSpread_avg'] = df['variableBorrowRate_avg']



# Export to a file for further analysis
df.to_csv("data.csv")

       timestamp  liquidityRate_avg  variableBorrowRate_avg  \
0     1726956000           0.021312                0.041170   
1     1726959600           0.021569                0.041595   
2     1726963200           0.021865                0.040993   
3     1726966800           0.021463                0.041480   
4     1726970400           0.021175                0.041833   
...          ...                ...                     ...   
1078  1728237600           0.059027                0.085799   
1079  1728241200           0.058570                0.085373   
1080  1728244800           0.057261                0.083723   
1081  1728248400           0.051088                0.074406   
1082  1728252000           0.049641                0.072082   

      utilizationRate_avg  stableBorrowRate_avg    chain currency  
0                0.612311                     0  polygon    SUSHI  
1                0.612347                     0  polygon    SUSHI  
2                0.612519              

In [116]:
json_rpc_url = aave_v3_get_json_rpc_url() or input("Please enter your JSON-RPC URL here")
web3 = Web3(HTTPProvider(json_rpc_url))

aave_network = aave_v3_get_network_by_chain_id(web3.eth.chain_id)

# Up to the current block
#max_block = web3.eth.block_number
max_block = 32_000_000 + 1_000_000

print(f'Detected network {aave_network.name } chain {web3.eth.chain_id} start block {aave_network.pool_created_at_block} max block {max_block}')

Detected network Polygon chain 137 start block 25826028 max block 33000000


In [119]:
start_block = 32_000_000  # Read from creation of the Aave v3 pool
end_block = max_block  # or Current block -> web3.eth.block_number
max_workers = 4  # number of workers to use for parallel API request processing

# Stores the last block number of event data we store
state = JSONFileScanState(f'aave-v3-{aave_network.name.lower()}-rate-scan.json')

aave_v3_fetch_events_to_csv(json_rpc_url, state, aave_network.name, start_block=start_block, end_block=end_block, max_workers=max_workers)

No previous scan done, starting fresh from block 32,000,000, total 1,000,000 blocks
Scanning block range 32,000,000 - 33,000,000


  0%|          | 0/1000000 [00:00<?, ?it/s]


RuntimeError: Could not decode {'address': '0x794a61358d6845594f94dc1db02a252b5b4814ad', 'blockHash': '0x8f9b2598b217244ab22aaaca69e4f60e0df29412c5c9a633259d2827c91900de', 'blockNumber': 32000002, 'data': '0x00000000000000000000000000000000000000000068ffe8966d9878f758d60b0000000000000000000000000000000000000000004a723dc6b40b8a9a00000000000000000000000000000000000000000000000108b447db9909de9af2909e00000000000000000000000000000000000000000342660ddf04ec3bcde3dde000000000000000000000000000000000000000000351a847f11d50b2dcfed4a9', 'logIndex': '0xac', 'removed': False, 'topics': ['0x804c9b842b2748a22bb64b345453a3de7ca54a6ca45ce00d415894979e22897a', '0x0000000000000000000000000d500b1d8e8ef31e21c99d1db9a6444d3adf1270'], 'transactionHash': '0x7aa75b8f03bf19ff47b771db83a24f66bf6c3d1f008294475ca897db244a9af8', 'transactionIndex': '0x2d', 'context': <eth_defi.aave_v3.events.TokenCache object at 0x37ebc6900>, 'event': <class 'web3._utils.datatypes.ReserveDataUpdated'>, 'chunk_id': 32000000, 'timesta

In [98]:
reserve_data_updated_df = pd.read_csv(f'/tmp/aave-v3-{aave_network.name.lower()}-reservedataupdated.csv')

# Index the dataset by timestamp (convert to datetime objects)
reserve_data_updated_df['timestamp'] = pd.to_datetime(reserve_data_updated_df['timestamp'])
reserve_data_updated_df.set_index('timestamp', drop=False, inplace=True)

print(f"We have total {len(reserve_data_updated_df):,} reserve data updates in the dataset")

We have total 0 reserve data updates in the dataset


In [124]:
# Initialize the Coingecko API client
cg = CoinGeckoAPI()

# Get current timestamp and calculate the timestamp for 30 days ago
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=30)

# Convert to 'dd-mm-yyyy' format as required by Coingecko
end_date_str = end_date.strftime('%d-%m-%Y')
start_date_str = start_date.strftime('%d-%m-%Y')

# Fetch historical price data for DAI (Polygon)
dai_history = cg.get_coin_market_chart_range_by_id(
    id='dai', 
    vs_currency='usd', 
    from_timestamp=start_date.timestamp(),
    to_timestamp=end_date.timestamp()
)

# Extract prices and timestamps
prices = dai_history['prices']

# Display price data (time is in milliseconds, so we convert it to a human-readable format)
for price_data in prices:
    timestamp = price_data[0] // 1000  # Convert from milliseconds to seconds
    price = price_data[1]
    date = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    print(f"Date: {date}, Price: {price} USD")

DAI/USD Price on Polygon via Chainlink: 4104565.65874253 USD


In [128]:
def process_data(dataset_path):
  """
  Processes lending data from a directory structure where folders represent chains and CSV files within each folder represent currencies.

  Args:
    dataset_path: The path to the root directory of the dataset. (default: ./dataset)

  Returns:
    A dataframe with columns: timestamp, liquidityRate_avg, variableBorrowRate_avg, utilizationRate_avg, stableBorrowRate_avg, 
  """

  all_data = []
  for chain in os.listdir(dataset_path):
    chain_path = os.path.join(dataset_path, chain)
    if os.path.isdir(chain_path):
      for filename in os.listdir(chain_path):
        if filename.endswith(".csv"):
          currency = filename[:-4]  # Remove '.csv' extension
          filepath = os.path.join(chain_path, filename)
          df = pd.read_csv(filepath)
          if df.empty: #Check if empty
            continue
          df['chain'] = chain  # Add chain column
          df['currency'] = currency  # Add currency column
          all_data.append(df)

  return pd.concat(all_data, ignore_index=True)

dataset_path = "./dataset"  
df = process_data(dataset_path)

# Sort values by timestap
df.sort_values('timestamp')

print(df)

# Export to a file for further analysis
df.to_csv("data.csv")

        timestamp  liquidityRate_avg  variableBorrowRate_avg  \
0      1726956000           0.015056                0.020055   
1      1726959600           0.023990                0.031854   
2      1726963200           0.014803                0.019608   
3      1726966800           0.016902                0.022262   
4      1726970400           0.014856                0.019768   
...           ...                ...                     ...   
13831  1728223200           0.000314                0.013196   
13832  1728226800           0.000110                0.004627   
13833  1728230400           0.000110                0.004627   
13834  1728234000           0.000110                0.004627   
13835  1728237600           0.000110                0.004627   

       utilizationRate_avg  stableBorrowRate_avg      chain currency  
0                 0.878280                     0   optimism     WETH  
1                 0.878118                     0   optimism     WETH  
2                 

In [11]:
# Sort values by timestap
df.sort_values('timestamp')

# Convert timestamp to datetime objects
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Format timestamp as day/month/year H:M
df['formatted_timestamp'] = df['timestamp'].dt.strftime('%d/%m/%Y %H:%M')

print(df)

                timestamp  liquidityRate_avg  variableBorrowRate_avg  \
0     2024-09-21 22:00:00           0.015056                0.020055   
1     2024-09-21 23:00:00           0.023990                0.031854   
2     2024-09-22 00:00:00           0.014803                0.019608   
3     2024-09-22 01:00:00           0.016902                0.022262   
4     2024-09-22 02:00:00           0.014856                0.019768   
...                   ...                ...                     ...   
13831 2024-10-06 14:00:00           0.000314                0.013196   
13832 2024-10-06 15:00:00           0.000110                0.004627   
13833 2024-10-06 16:00:00           0.000110                0.004627   
13834 2024-10-06 17:00:00           0.000110                0.004627   
13835 2024-10-06 18:00:00           0.000110                0.004627   

       utilizationRate_avg  stableBorrowRate_avg      chain currency  \
0                 0.878280                     0   optimism    

In [ ]:
# For each folder in dataset, open every folder


Now we are gonna use the 

The EMA Bearish Crossover is a technical analysis signal that indicates a potential downward trend or selling opportunity. It occurs when a short-term Exponential Moving Average (EMA) crosses below a longer-term EMA. This crossover suggests that the momentum is shifting from bullish (upward) to bearish (downward), which traders often interpret as a sell signal.

In [ ]:
#Calculate the short term (10 days) EMA Bearish Crossover 

In [ ]:
#Calculate the long term (100 days) EMA Bearish Crossover

If the 10-day EMA has crossed below the 100-day EMA, you could interpret it as a potential sell signal.
You then check the ADX:
-If ADX is above 25, you know the downtrend is strong, confirming the bearish move, and it may be a good opportunity to sell or reduce exposure.
-If ADX is below 20, the crossover might not be reliable, and you might decide to wait for more confirmation before acting.

In [ ]:
#Calculate the short term (10 days) ADX 

In [ ]:
#Calculate the short term (10 days) ADX 